In [24]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Your feature engineering function (unchanged)
def engineer_features_no_pca(df):
    df = df.copy()
    new_cols = {}

    # Weighted component-property features
    for comp in range(1, 6):
        for prop in range(1, 11):
            prop_col = f'Component{comp}_Property{prop}'
            frac_col = f'Component{comp}_fraction'
            new_cols[f'{prop_col}_weighted'] = df[prop_col] * df[frac_col]

    # Interaction features between fractions
    for i in range(1, 6):
        frac_i = f'Component{i}_fraction'
        for j in range(i + 1, 6):
            frac_j = f'Component{j}_fraction'
            new_cols[f'{frac_i}_x_{frac_j}'] = df[frac_i] * df[frac_j]
            new_cols[f'{frac_i}_div_{frac_j}'] = df[frac_i] / (df[frac_j] + 1e-8)

    # Ratios to total fraction
    total_fraction = sum(df[f'Component{i}_fraction'] for i in range(1, 6))
    new_cols['Total_fraction'] = total_fraction
    for i in range(1, 6):
        frac_col = f'Component{i}_fraction'
        new_cols[f'{frac_col}_to_total'] = df[frac_col] / (total_fraction + 1e-8)

    # Aggregate stats for component properties
    for comp in range(1, 6):
        props = [f'Component{comp}_Property{p}' for p in range(1, 11)]
        new_cols[f'Component{comp}_prop_mean'] = df[props].mean(axis=1)
        new_cols[f'Component{comp}_prop_std'] = df[props].std(axis=1)
        new_cols[f'Component{comp}_prop_min'] = df[props].min(axis=1)
        new_cols[f'Component{comp}_prop_max'] = df[props].max(axis=1)
        new_cols[f'Component{comp}_prop_range'] = new_cols[f'Component{comp}_prop_max'] - new_cols[f'Component{comp}_prop_min']

    # Polynomial features for component fractions (squares)
    for i in range(1, 6):
        frac_col = f'Component{i}_fraction'
        new_cols[f'{frac_col}_squared'] = df[frac_col] ** 2

    # Cross product polynomial features
    for i in range(1, 6):
        frac_i = f'Component{i}_fraction'
        for j in range(i + 1, 6):
            frac_j = f'Component{j}_fraction'
            new_cols[f'{frac_i}_x_{frac_j}'] = df[frac_i] * df[frac_j]

    new_features_df = pd.DataFrame(new_cols, index=df.index)
    df = pd.concat([df, new_features_df], axis=1)
    return df
# Load data
X = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/X_train.csv')
y = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/y_train.csv')
X_test = pd.read_csv('/Users/xDAyN/Desktop/cs project/Project Shell.ai Hackathon/dataset/X_test.csv')

# Feature engineering
X_fe = engineer_features_no_pca(X)
X_test_fe = engineer_features_no_pca(X_test)

# Target 7 = index 6
target_index = 6

# Train/val split
X_train, X_val, y_train, y_val = train_test_split(X_fe, y, test_size=0.2, random_state=42)

# Define and train model
model = xgb.XGBRegressor(
    max_depth=5,
    n_estimators=100000,
    learning_rate=0.05,
    reg_alpha=0,
    reg_lambda=5.1,
    eval_metric='mae',
    early_stopping_rounds=50,
    random_state=42,
    verbosity=1
)

print(f"\nTraining XGBoost on Target {target_index + 1} using ALL features...")
model.fit(
    X_train, y_train.iloc[:, target_index],
    eval_set=[(X_val, y_val.iloc[:, target_index])],
    verbose=50
)

# Predict
train_preds = model.predict(X_train)
val_preds = model.predict(X_val)

# Evaluation
train_mae = mean_absolute_error(y_train.iloc[:, target_index], train_preds)
val_mae = mean_absolute_error(y_val.iloc[:, target_index], val_preds)
train_mape = mean_absolute_percentage_error(y_train.iloc[:, target_index], train_preds)
val_mape = mean_absolute_percentage_error(y_val.iloc[:, target_index], val_preds)

print(f"\nTrain MAE: {train_mae:.5f}")
print(f"Validation MAE: {val_mae:.5f}")
print(f"Train MAPE: {train_mape:.5f}")
print(f"Validation MAPE: {val_mape:.5f}")

# Feature importance
booster = model.get_booster()
importance = booster.get_score(importance_type='total_gain')
importance_df = pd.DataFrame({
    'feature': list(importance.keys()),
    'total_gain': list(importance.values())
}).sort_values(by='total_gain', ascending=False)
# --- Predict on test set using all features ---
test_preds = model.predict(X_test_fe)

# Create DataFrame for test predictions
test_output = pd.DataFrame(test_preds, columns=[f'Target{target_index + 1}_prediction'])

# Print first few predictions
print(f"\nTest Predictions for Target {target_index + 1}:")
print(test_output.head())

# Save to CSV
test_output.to_csv(f'test_predictions_target_{target_index + 1}.csv', index=False)
print(f"\nTest predictions saved to 'test_predictions_target_{target_index + 1}.csv'")



Training XGBoost on Target 7 using ALL features...
[0]	validation_0-mae:0.75977
[50]	validation_0-mae:0.21831
[100]	validation_0-mae:0.15666
[150]	validation_0-mae:0.14171
[200]	validation_0-mae:0.13639
[250]	validation_0-mae:0.13375
[300]	validation_0-mae:0.13303
[350]	validation_0-mae:0.13237
[400]	validation_0-mae:0.13192
[450]	validation_0-mae:0.13199
[500]	validation_0-mae:0.13163
[550]	validation_0-mae:0.13158
[600]	validation_0-mae:0.13137
[650]	validation_0-mae:0.13130
[700]	validation_0-mae:0.13123
[750]	validation_0-mae:0.13131
[771]	validation_0-mae:0.13135

Train MAE: 0.00924
Validation MAE: 0.13120
Train MAPE: 0.09445
Validation MAPE: 0.49297

Test Predictions for Target 7:
   Target7_prediction
0            0.609912
1           -1.256830
2            0.959617
3            0.526026
4            1.121560

Test predictions saved to 'test_predictions_target_7.csv'
